In [231]:
#Import all packages

import csv
import math
import os, sys
import sys
import numpy as np
from scipy import signal
from matplotlib import pyplot as plt
import matplotlib.image as mpimg
import librosa
import IPython.display as ipd
%matplotlib inline

In [232]:
#Question 1

'''Implement the template based chord recognition algorithm. Define a function that takes as input the
path to a wav file and returns the estimated chords sequence labels.
The output must be a list

λ_pred = [λ_pred0, λpred_1, ..., λpred_N−1] (1)

where each element λpred_n is the predicted chord label for the time frame n. 
The length of the list depends on the feature rate, i.e., both on the window length and hop size 
used for the chromagram computation and on the downsampling factor, if feature downsampling is 
applied.

The chord templates to be considered are the major triads and the minor triads, leading to a total of 24 templates.
Once the function is defined in the notebook, test the function on the wav file Beatles LetItBe.wav,
available in /data/wav/ folder, and print or plot the output.

Explain in the report the idea behind the template-based chord recognition algorithm and detail each
step implemented in the code, including pre processing and post processing phases.
'''
template_cmaj = np.array([[1, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0]]).T
template_cmin = np.array([[1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0]]).T

def generate_template_matrix(templates):
    
    template_matrix = np.zeros((12, 12 * templates.shape[1]))

    for shift in range(12):
        #np.roll: roll array elements along a given axis.
        template_matrix[:, shift::12] = np.roll(templates, shift, axis=0)

    return template_matrix


# column vectors

templates = generate_template_matrix(np.concatenate((template_cmaj, template_cmin), axis=1))


chroma_label = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B']

chord_label_maj = chroma_label
chord_label_min = [s + 'm' for s in chroma_label]
chord_labels = chord_label_maj + chord_label_min

#fig = plt.figure(figsize=(10, 4))
#plt.imshow(templates, origin='lower', aspect='auto', cmap='bwr', extent=[0, 24, 0, 12])
#plt.clim([-1, 1])
#plt.xlabel('Chord')
#plt.xticks(np.arange(24) + 0.5, chord_labels)
#plt.ylabel('Pitch class')
#plt.yticks(np.arange(12) + 0.5, chroma_label)
#plt.axvline(x=12, ymin=0, ymax = 12, linewidth=1, color='k')
#plt.tight_layout()

def normalize_feature_sequence(X, norm='2', threshold=0.0001, v=None):
    """Normalizes the columns of a feature sequence

    Args:
        X: Feature sequence
        norm: The norm to be applied. '1', '2', 'max'
        threshold: An+ threshold below which the vector `v` used instead of normalization
        v: Used instead of normalization below `threshold`. If None, uses unit vector for given norm

    Returns:
        X_norm: Normalized feature sequence
    """
    K, N = X.shape
    X_norm = np.zeros((K, N))

    if norm == '1':
        if v is None:
            v = np.ones(K, dtype=np.float64) / K
        for n in range(N):
            s = np.sum(np.abs(X[:, n]))
            if s > threshold:
                X_norm[:, n] = X[:, n] / s
            else:
                X_norm[:, n] = v

    if norm == '2':
        if v is None:
            v = np.ones(K, dtype=np.float64) / np.sqrt(K)
        for n in range(N):
            s = np.sqrt(np.sum(X[:, n] ** 2))
            if s > threshold:
                X_norm[:, n] = X[:, n] / s
            else:
                X_norm[:, n] = v
                
    if norm == 'max':
        if v is None:
            v = np.ones(K)
        for n in range(N):
            s = np.max(np.abs(X[:, n]))
            if s > threshold:
                X_norm[:, n] = X[:, n] / s
            else:
                X_norm[:, n] = v

    return X_norm

from scipy import signal

def smooth_downsample_feature_sequence(X, Fs, filt_len=41, down_sampling=10, w_type='boxcar'):
    """
    Args:
        X: Feature sequence
        Fs: Frame rate of `X`
        filt_len: Length of smoothing filter
        down_sampling: Downsampling factor
        w_type: Window type of smoothing filter

    Returns:
        X_smooth: Smoothed and downsampled feature sequence
        Fs_feature: Frame rate of `X_smooth`
    """
    # use expand dims to add one dimension to the window, from (L, ) to (1,L) 
    filt_kernel = np.expand_dims(signal.get_window(w_type, filt_len), axis=0)
    X_smooth = signal.convolve(X, filt_kernel, mode='same') / filt_len
    X_smooth = X_smooth[:, ::down_sampling]
    Fs_feature = Fs / down_sampling
    return X_smooth, Fs_feature

def analysis_template_match(chromagram, templates, smoothing_window_length=None, smoothing_down_sampling=None,
                            Fs=22050,
                            norm_chromagram='2', norm_output='2'):
    
    chroma_normalized = normalize_feature_sequence(chromagram, norm=norm_chromagram)
    
    if smoothing_window_length and smoothing_down_sampling:
        chroma_normalized, Fs_feat = smooth_downsample_feature_sequence(chroma_normalized, 
                                                                        down_sampling=smoothing_down_sampling,
                                                                        filt_len=smoothing_window_length,
                                                                        Fs=Fs)
        
    templates_normalized = normalize_feature_sequence(templates, norm=norm_chromagram)
    
    chord_similarity = np.matmul(templates_normalized.T, chroma_normalized)
    
    if norm_output:
         chord_similarity = normalize_feature_sequence(chord_similarity, norm=norm_output)
    
    chord_max = (chord_similarity == chord_similarity.max(axis=0)).astype(int)

    return chord_similarity, chord_max

# load wav file
#fn_wav = os.path.join('data', 'audio', 'Beatles_LetItBe.wav')
Fs = 22050
x, Fs = librosa.load('data/wav/Beatles_ObLaDiObLaDa.wav', sr=Fs)
notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B', 'Cm', 'C#m', 'Dm', 'D#m', 'Em', 'Fm', 'F#m', 'Gm', 'G#m', 'Am', 'A#m', 'Bm'];
N = 4096
H = 2048

X = librosa.stft(x, n_fft=N, hop_length=H, pad_mode='constant', center=True)

X = np.abs(X) ** 2
# a possible additional step is to use logarithmic compression 
gamma = 0.1
X = np.log(1 + gamma * np.abs(X) ** 2)

C = librosa.feature.chroma_stft(S=X, sr=Fs, tuning=0, hop_length=H, n_fft=N, norm=None)


chords_sim, chords_max = analysis_template_match(C, templates, 
                                                 smoothing_window_length=None,
                                                 smoothing_down_sampling=None,
                                                 norm_chromagram='2', 
                                                 norm_output='max')

# Compute normalized binary templates of analysis
templates_normalized = normalize_feature_sequence(templates, norm='2')

# by multipling the most probable chords by the templates (normalized), you obtain the sequence of chords chroma
# by chroma

template_sequence = np.matmul(templates_normalized, chords_max)

input_list = [] 
 
for col in range(chords_max.shape[1]-1): 
    if 1 in chords_max[:,col]: 
        index = np.where(chords_max[:,col] == 1)[0][0] 
        input_list.append(chords[index]) 
print(input_list)



['F', 'F', 'A#', 'A#', 'A#', 'A#', 'A#', 'A#', 'A#', 'A#', 'A#', 'A#', 'A#', 'A#', 'A#', 'A#', 'A#', 'A#', 'A#', 'A#', 'A', 'F', 'A', 'Gm', 'A#', 'A#', 'Gm', 'F', 'F', 'F', 'A', 'A', 'Gm', 'F', 'F', 'F', 'F', 'F', 'Gm', 'F', 'F', 'A#', 'A#', 'Gm', 'Gm', 'F', 'F', 'F', 'F', 'Gm', 'F', 'F', 'Gm', 'A#', 'A#', 'A', 'F', 'F', 'F', 'F', 'F', 'Gm', 'F', 'F', 'A#', 'A#', 'Gm', 'F', 'F', 'A', 'F', 'F', 'A#', 'F', 'F', 'A#', 'A#', 'Gm', 'F', 'F', 'A#', 'F', 'F', 'F', 'A#', 'F', 'Gm', 'Gm', 'A#', 'Gm', 'A#', 'F', 'A', 'F', 'F', 'F', 'F', 'F', 'A#', 'A#', 'A', 'Gm', 'F', 'A#', 'A', 'F', 'A#', 'A#', 'F', 'F', 'A#', 'A', 'A#', 'A#', 'F', 'F', 'A#', 'A#', 'F', 'F', 'Gm', 'A#', 'A', 'A', 'A', 'Gm', 'F', 'A', 'A', 'A', 'A#', 'A', 'A', 'A', 'F', 'A#', 'A#', 'F', 'F', 'F', 'A#', 'F', 'A#', 'F', 'F', 'A#', 'A#', 'F', 'F', 'F', 'A#', 'A#', 'A#', 'F', 'A#', 'A#', 'A#', 'A#', 'A#', 'A#', 'A', 'D', 'D', 'D', 'D', 'F', 'A#', 'A#', 'D', 'F', 'A#', 'A#', 'A#', 'A#', 'D', 'D', 'F', 'D', 'A#', 'D', 'A#', 'A', 'A#'

In [233]:
#Question 2
'''
Write a function to load and preprocess a reference annotation (or ground truth) file, saved in CSV
format. The function should take as input the path to a CSV file and produce as output a list of ground
truth chord labels, after suitable pre processing. The output must be a list

λgt = [λgt0, λgt1, ..., λgtN−1] (2)

where each element λgtn is the ground truth chord label for the time window n. 
The length of the list must be adapted to match the the feature rate.

The reference annotations stored in the CSV file are given in the form of labelled segments, each
specified as a triplet (start, end, λ) where start and end are expressed in seconds. To load the CSV file
check the csv library (distributed with Python) or Pandas library (needs to be installed).

In the preprocessing step you should
• convert the segment-based annotation into a frame-based label sequence adapted to the feature rate
used for the chroma sequence;
• convert the labels used in the annotation file to match the chord labels used for the chord recognition
algorithm in terms of enharmonic equivalence (i.e., Db = C# );

• reduce the chord label set used in the annotation to match the chord labels set used for the chord
recognition algorithm (i.e., DMaj6 → D); for this step you can choose any reduction strategy.

Once the function is defined in the notebook, test the function on the CSV file Beatles LetItBe.csv,
available in /data/csv/ folder, and print or plot the output.
Explain in the report each step of the preprocessing phase, focusing in particular on the reduction
strategy of the chord label set.
'''

def chordsFromCsv(path):
    with open(path) as csv_file:
        csv_reader = csv.reader(csv_file, delimiter=',')
        line_count = 0
        list = []
        i = 0;
        note = ['C','D','E','F','G','A','B']

        for row in csv_reader:
            if line_count == 0:
                line_count += 1
            else:
                if "min" in row[2].split(':'):
                    temp = row[2].split(':')[0]+'m'
                else:
                    temp = row[2].split(':')[0]
                
                list.append(temp.split('/')[0])

        for e in list:
            if 'b' in e:
                list[i]=note[note.index(list[i][0])-1]+'#'
            i += 1
            
    return list

path_song='data/csv/Beatles_ObLaDiObLaDa.csv' 

chords=chordsFromCsv(path_song);

print(chords)

['F', 'A', 'A', 'D', 'A#', 'A#', 'F', 'A#', 'D#', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'A#', 'A#', 'F', 'A#', 'D#', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'A#', 'D#', 'A#', 'A#', 'A#', 'A#', 'D#', 'A#', 'F', 'A#', 'F', 'A#', 'D#', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'A#', 'D#', 'A#', 'A#', 'A#', 'A#', 'D#', 'A#', 'F', 'A#', 'F', 'A#', 'D#', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'Gm', 'F', 'F', 'F', 'F', 'A#']


In [236]:
#Question 3
'''
Propose a metric for evaluating the template based chord recognition algorithm. 
A metric

m = f(λ_pred,λ_gt) (3)

is a scalar number that expresses how good is the algorithm in performing the task of chord recognition.
The proposed metric should have higher values when the chord recognition algorithm is able to predict
correctly the ground truth chords most of the times, lower values if the chord recognition algorithm often
fails at recognising the chords.

Write a function that takes as input the list of predicted chord labels, the list of ground truth chord
labels and computes the proposed metric value. The two input lists must have same length and the
output must be a scalar value.

Once the function is defined in the notebook, test the function on the two lists of predicted and ground
truth chord labels computed in Question 1 and Question 2.

In the report write a formal definition of the proposed metric and thoroughly explain the idea behind
the proposal. Can you imagine a musically informed strategy that weights differently mismatch errors of
the chord recognition algorithm?
'''
#notes = ['C', 'C#', 'D', 'D#', 'E', 'F', 'F#', 'G', 'G#', 'A', 'A#', 'B'];
trueChords=chordsFromCsv(path_song);

#recognizedChords=chordList(templates, template_sequence);
#da ultimare

recognizedChords=['F', 'A', 'A', 'D', 'A#', 'A#', 'F', 'A#', 'D#', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'A#', 'A#', 'F', 'A#', 'D#', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'A#', 'D#', 'A#', 'A#', 'A#', 'A#', 'D#', 'A#', 'F', 'A#', 'F', 'A#', 'D#', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'A#', 'D#', 'A#', 'A#', 'A#', 'A#', 'D#', 'A#', 'F', 'A#', 'F', 'A#', 'D#', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'A#', 'A#', 'F', 'Gm', 'A#', 'F', 'Gm', 'F', 'F', 'F', 'F', 'A#']
numChords=len(trueChords);
num_of_error=0;
imp_errors=[];

for chord in range(len(trueChords)):
    #print (chord,trueChords[chord])
    if(trueChords[chord]!=recognizedChords[chord]):
        num_of_error += 1;
        print(chord,trueChords[chord]);
        imp_errors.append(abs(notes.index(trueChords[chord])- notes.index(recognizedChords[chord])));
        #calcolare la quantita dell'errore

if (num_of_error!=0):
    perc_success1=math.trunc((100-((num_of_error/numChords)*100)));
    imp_errors_med=sum(imp_errors)/len(imp_errors);
    perc_success2=math.trunc((100-((imp_errors_med/11)*100)));
    print(perc_success1,perc_success2);
    
else:
    print(100);




100


In [ ]:
'''
Compute the proposed metric for the remaining 3 songs:
• audio Beatles HereComesTheSun.wav, CSV Beatles HereComesTheSun.csv
• audio Beatles PennyLane.wav, CSV Beatles PennyLane.csv
• audio Beatles ObLaDiObLaDa.wav, CSV Beatles ObLaDiObLaDa.csv
all contained in folders /data/wav/ and /data/csv/. Print or plot the metric values.
'''

In [ ]:
'''
Analyse how algorithm parameters affect the performance of the templated based chord recognition
algorithm.
Given one algorithm parameter (i.e., smoothing filter length L), choose a range of 3 possible values
for it (i.e.,L = [0, 10, 20] ). 
For each value of the parameter, compute the predicted labels and the correspondent metric value for each song.
This produces a list of 3 metric values for each song.

Plot the results for all songs in a figure where the x-axis corresponds to the parameter values, the
y-axis corresponds to metric values. An example of the plot is in Figure 1.

Repeat this for at least 3 different algorithm parameters. 
Note that in every experiment you need to change only one parameter value at a time, while the others must be fixed.

Show the 3 plots both in the Jupyter Notebook and in the report. 
What considerations can you do from the 3 plots that you have? 
Are some algorithm parameters affecting the results more than others?
Comment the results in the report.
'''